In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [30]:


zip_path = '/content/drive/MyDrive/ISIC_2024_Training_Input.zip'
extract_path = '/content/drive/MyDrive/ISIC_2024_Training_Input'

if os.path.exists(zip_path):
    file_size_gb = os.path.getsize(zip_path) / (1024**3)
    print(f" Found zip file: {zip_path}")
    print(f"   Size: {file_size_gb:.2f} GB")
else:
    print(f" Zip file not found: {zip_path}")
    print("Please check if the file name is exactly 'ISIC_2024_Training_Input.zip'")


 STEP 2: Verifying Zip File...
 Found zip file: /content/drive/MyDrive/ISIC_2024_Training_Input.zip
   Size: 1.15 GB


In [20]:


# Check if already extracted
extract_path = '/content/drive/MyDrive/ISIC_2024_Training_Input'

if os.path.exists(extract_path):
    existing_files = [f for f in os.listdir(extract_path) if f.endswith('.jpg')]
    print(f"Directory already exists with {len(existing_files):,} images")

    if len(existing_files) > 300000:  # Should have ~400k images
        print(" Images already fully extracted!")
    else:
        print(" Incomplete extraction, re-extracting...")
        # Remove incomplete directory
        !rm -rf "/content/drive/MyDrive/ISIC_2024_Training_Input"
else:
    print(" Starting fresh extraction...")

# Extract using unzip command (more reliable than Python zipfile)
if not os.path.exists(extract_path) or len(os.listdir(extract_path)) < 300000:
    print(" Extracting... this will take 5-10 minutes")
    print(" Perfect time for a coffee break!")


    # Use unzip command - more reliable for large files
    !cd "/content/drive/MyDrive" && unzip -q "ISIC_2024_Training_Input.zip"

    # Verify extraction
    if os.path.exists(extract_path):
        extracted_files = [f for f in os.listdir(extract_path) if f.endswith('.jpg')]
        print(f" Successfully extracted {len(extracted_files):,} images!")
    else:
        print(" Extraction failed")


 STEP 3: Extracting Images (Command Line Method)...
 Starting fresh extraction...
 Extracting... this will take 5-10 minutes
 Perfect time for a coffee break!
 Successfully extracted 184,991 images!


In [21]:
GROUNDTRUTH_PATH = '/content/drive/MyDrive/ISIC_2024_Training_GroundTruth.csv'
SUPPLEMENT_PATH = '/content/drive/MyDrive/ISIC_2024_Training_Supplement.csv'
METADATA_PATH = '/content/drive/MyDrive/metadata.csv'
IMAGE_DIR = '/Users/raielamesquita/Desktop/ISIC_2024_Training_Input'  # Your 400k images folder

# Training configuration (following your project approach)
SAMPLE_SIZE = 50000      # Start with manageable subset
BATCH_SIZE = 32          # Efficient for EfficientNet
IMG_SIZE = (224, 224)    # EfficientNet-B0 input size
EPOCHS = 30             # Sufficient for convergence
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 1e-4    # Conservative for transfer learning

print(f" Data Configuration:")
print(f"   • Sample size: {SAMPLE_SIZE:,} images")
print(f"   • Image size: {IMG_SIZE}")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Learning rate: {LEARNING_RATE}")

 Data Configuration:
   • Sample size: 50,000 images
   • Image size: (224, 224)
   • Batch size: 32
   • Learning rate: 0.0001


In [22]:
#data processing

# Load all three datasets as per your structure
groundtruth_df = pd.read_csv(GROUNDTRUTH_PATH)
supplement_df = pd.read_csv(SUPPLEMENT_PATH)
metadata_df = pd.read_csv(METADATA_PATH)

print(f" Ground truth: {groundtruth_df.shape}")
print(f" Supplement: {supplement_df.shape}")
print(f" Metadata: {metadata_df.shape}")

# Merge datasets (following your methodology)
df = metadata_df.merge(groundtruth_df, on='isic_id', how='inner')
print(f" Merged dataset: {df.shape}")

# Data cleaning (as described in your milestone)
print(f"\n Data Cleaning (Following Your Milestone Approach):")
initial_size = len(df)

# Check missing values for key columns
key_columns = ['sex', 'anatom_site_general', 'age_approx']
missing_summary = {}
for col in key_columns:
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    missing_summary[col] = {'count': missing_count, 'pct': missing_pct}
    print(f"   • {col}: {missing_count:,} missing ({missing_pct:.2f}%)")

# Drop rows with missing key values (your approach)
df_clean = df.dropna(subset=key_columns).copy()
retention_rate = (len(df_clean) / initial_size) * 100
print(f" After cleaning: {len(df_clean):,} samples ({retention_rate:.2f}% retained)")

# Create binary features as described in your milestone
df_clean['is_60_plus'] = (df_clean['age_approx'] >= 60).astype(int)
df_clean['is_head_neck'] = (df_clean['anatom_site_general'] == 'head/neck').astype(int)

print(f"✓ Binary features created:")
print(f"   • is_60_plus: {df_clean['is_60_plus'].sum():,} patients ≥60 years")
print(f"   • is_head_neck: {df_clean['is_head_neck'].sum():,} head/neck lesions")

# Sample data for training
df_sample = df_clean.sample(n=min(SAMPLE_SIZE, len(df_clean)), random_state=42)

print(f"\n Dataset Statistics:")
print(f"   • Final sample size: {len(df_sample):,}")
print(f"   • Malignancy rate: {df_sample['malignant'].mean():.4f}")
print(f"   • Class imbalance ratio: {(1-df_sample['malignant'].mean())/df_sample['malignant'].mean():.1f}:1")
print(f"   • Unique patients: {df_sample['patient_id'].nunique():,}")

 Ground truth: (401059, 2)
 Supplement: (401059, 13)
 Metadata: (401059, 42)
 Merged dataset: (401059, 43)

 Data Cleaning (Following Your Milestone Approach):
   • sex: 11,517 missing (2.87%)
   • anatom_site_general: 5,756 missing (1.44%)
   • age_approx: 2,798 missing (0.70%)
 After cleaning: 381,914 samples (95.23% retained)
✓ Binary features created:
   • is_60_plus: 201,756 patients ≥60 years
   • is_head_neck: 11,688 head/neck lesions

 Dataset Statistics:
   • Final sample size: 50,000
   • Malignancy rate: 0.0011
   • Class imbalance ratio: 908.1:1
   • Unique patients: 975


In [23]:
#efficientnet-b0 with transfer learning

def create_efficientnet_cnn(input_shape=(224, 224, 3), num_classes=1):

    # Load pre-trained EfficientNet-B0 (ImageNet weights)
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model initially
    base_model.trainable = False

    # Add custom classification head
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='sigmoid')
    ], name='EfficientNet_B0_SkinCancer')

    return model

def create_multimodal_efficientnet(input_shape=(224, 224, 3), metadata_features=2):

    # Image branch - EfficientNet-B0
    image_input = layers.Input(shape=input_shape, name='image_input')
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=image_input)
    base_model.trainable = False

    # Image feature extraction
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    image_features = layers.Dense(512, activation='relu')(x)

    # Tabular data branch (your binary features)
    tabular_input = layers.Input(shape=(metadata_features,), name='tabular_input')
    tabular_features = layers.Dense(64, activation='relu')(tabular_input)
    tabular_features = layers.BatchNormalization()(tabular_features)
    tabular_features = layers.Dense(32, activation='relu')(tabular_features)

    # Fusion layer (concatenation as described in your milestone)
    combined = layers.concatenate([image_features, tabular_features])
    combined = layers.BatchNormalization()(combined)
    combined = layers.Dropout(0.5)(combined)

    # Final classification layers
    combined = layers.Dense(256, activation='relu')(combined)
    combined = layers.Dropout(0.3)(combined)
    combined = layers.Dense(128, activation='relu')(combined)
    combined = layers.Dropout(0.2)(combined)

    # Output layer
    output = layers.Dense(1, activation='sigmoid', name='malignancy_prediction')(combined)

    model = keras.Model(inputs=[image_input, tabular_input], outputs=output,
                       name='Multimodal_EfficientNet_SkinCancer')
    return model

# Build models
print("Building EfficientNet-B0 models...")

# 1. Image-only CNN model
cnn_model = create_efficientnet_cnn()
print(f" EfficientNet-B0 CNN created")

# 2. Multimodal CNN model (images + tabular features)
multimodal_cnn = create_multimodal_efficientnet(metadata_features=2)
print(f" Multimodal EfficientNet-B0 created")

# Display model architectures
print(f"\n EfficientNet-B0 CNN Architecture:")
cnn_model.summary()

print(f"\n Multimodal EfficientNet-B0 Architecture:")
multimodal_cnn.summary()

Building EfficientNet-B0 models...
 EfficientNet-B0 CNN created
 Multimodal EfficientNet-B0 created

 EfficientNet-B0 CNN Architecture:


Model: "EfficientNet_B0_SkinCancer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,844,196 (18.48 MB)

 Trainable params: 791,041 (3.02 MB)

...

Model: "Multimodal_EfficientNet_SkinCancer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_10        │ (None, 224, 224,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_5     │ (None, 224, 224,  │          7 │ rescaling_10[0][… │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_11        │ (None, 224, 224,  │          0 │ normalization_5[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_11[0][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,887,812 (18.65 MB)

 Trainable params: 834,465 (3.18 MB)

 Non-trainable params: 4,053,347 (15.46 MB)

In [24]:
#focal loss implementation


def focal_loss(gamma=2.0, alpha=0.75):

    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Calculate focal loss
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_loss = -alpha_t * tf.pow((1 - p_t), gamma) * tf.math.log(p_t)

        return tf.reduce_mean(focal_loss)

    return focal_loss_fixed

# Calculate class weights as backup
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(df_sample['malignant']),
    y=df_sample['malignant']
)
class_weight_dict = dict(zip([0, 1], class_weights))
print(f"✓ Class weights calculated: {class_weight_dict}")

# Compile models with focal loss
print("Compiling models with focal loss...")

cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=['accuracy', 'precision', 'recall', 'AUC']
)

multimodal_cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=['accuracy', 'precision', 'recall', 'AUC']
)

print(" Models compiled with focal loss (γ=2, α=0.75)")

✓ Class weights calculated: {0: np.float64(0.5005506056662329), 1: np.float64(454.54545454545456)}
Compiling models with focal loss...
 Models compiled with focal loss (γ=2, α=0.75)


In [28]:
def validate_available_images(df, image_dir):
    """
    Check which images actually exist and filter the dataset accordingly
    """
    print(f"Checking image availability in: {image_dir}")

    available_images = []
    missing_images = []

    # Check if image directory exists
    if not os.path.exists(image_dir):
        print(f" Image directory not found: {image_dir}")
        print("Please check your IMAGE_DIR path!")
        return df.iloc[:0]  # Return empty dataframe

    # Check each image
    for idx, row in df.iterrows():
        image_path = os.path.join(image_dir, f"{row['isic_id']}.jpg")
        if os.path.exists(image_path):
            available_images.append(idx)
        else:
            missing_images.append(row['isic_id'])

        if len(available_images) % 5000 == 0:
            print(f"   Checked {len(available_images) + len(missing_images):,} images...")

    # Filter to only available images
    df_available = df.loc[available_images].copy()

    print(f" Image validation completed:")
    print(f"   • Total images in metadata: {len(df):,}")
    print(f"   • Available images: {len(df_available):,}")
    print(f"   • Missing images: {len(missing_images):,}")
    print(f"   • Availability rate: {len(df_available)/len(df)*100:.1f}%")

    if len(missing_images) > 0:
        print(f"   • First few missing: {missing_images[:5]}")

    return df_available

# Validate images before proceeding
df_sample_validated = validate_available_images(df_sample, IMAGE_DIR)

if len(df_sample_validated) == 0:
    print(" No images found! Please check your IMAGE_DIR path.")
    print(f"Current IMAGE_DIR: {IMAGE_DIR}")
    print("Make sure you've unzipped the ISIC_2024_Training_Input.zip file.")
else:
    print(f" Proceeding with {len(df_sample_validated):,} validated images")
    df_sample = df_sample_validated  # Update the sample to use only available images


# Patient-level splitting to prevent data leakage (critical for medical ML)
unique_patients = df_sample['patient_id'].unique()
train_patients, val_patients = train_test_split(
    unique_patients,
    test_size=VALIDATION_SPLIT,
    random_state=42
)

# Create train and validation sets
train_df = df_sample[df_sample['patient_id'].isin(train_patients)]
val_df = df_sample[df_sample['patient_id'].isin(val_patients)]

print(f"✓ Patient-level split completed:")
print(f"   • Training: {len(train_patients):,} patients ({len(train_df):,} images)")
print(f"   • Validation: {len(val_patients):,} patients ({len(val_df):,} images)")
print(f"   • Train malignancy rate: {train_df['malignant'].mean():.4f}")
print(f"   • Validation malignancy rate: {val_df['malignant'].mean():.4f}")

# Prepare tabular features for multimodal model
train_tabular = train_df[['is_60_plus', 'is_head_neck']].values.astype(np.float32)
val_tabular = val_df[['is_60_plus', 'is_head_neck']].values.astype(np.float32)

print(f"✓ Tabular features prepared: {train_tabular.shape}")


 STEP 4.5: Validating Available Images...
Checking image availability in: /Users/raielamesquita/Desktop/ISIC_2024_Training_Input
 Image directory not found: /Users/raielamesquita/Desktop/ISIC_2024_Training_Input
Please check your IMAGE_DIR path!
 No images found! Please check your IMAGE_DIR path.
Current IMAGE_DIR: /Users/raielamesquita/Desktop/ISIC_2024_Training_Input
Make sure you've unzipped the ISIC_2024_Training_Input.zip file.

 STEP 4: Patient-Level Data Splitting...
✓ Patient-level split completed:
   • Training: 780 patients (41,190 images)
   • Validation: 195 patients (8,810 images)
   • Train malignancy rate: 0.0012
   • Validation malignancy rate: 0.0008
✓ Tabular features prepared: (41190, 2)


In [25]:
#patient level data splitting


# Patient-level splitting to prevent data leakage
unique_patients = df_sample['patient_id'].unique()
train_patients, val_patients = train_test_split(
    unique_patients,
    test_size=VALIDATION_SPLIT,
    random_state=42
)

# Create train and validation sets
train_df = df_sample[df_sample['patient_id'].isin(train_patients)]
val_df = df_sample[df_sample['patient_id'].isin(val_patients)]

print(f" Patient-level split completed:")
print(f"   • Training: {len(train_patients):,} patients ({len(train_df):,} images)")
print(f"   • Validation: {len(val_patients):,} patients ({len(val_df):,} images)")
print(f"   • Train malignancy rate: {train_df['malignant'].mean():.4f}")
print(f"   • Validation malignancy rate: {val_df['malignant'].mean():.4f}")

# Prepare tabular features for multimodal model
train_tabular = train_df[['is_60_plus', 'is_head_neck']].values.astype(np.float32)
val_tabular = val_df[['is_60_plus', 'is_head_neck']].values.astype(np.float32)

print(f"✓ Tabular features prepared: {train_tabular.shape}")

 Patient-level split completed:
   • Training: 780 patients (41,190 images)
   • Validation: 195 patients (8,810 images)
   • Train malignancy rate: 0.0012
   • Validation malignancy rate: 0.0008
✓ Tabular features prepared: (41190, 2)


In [26]:
#efficient data pipeline with augmentation


def create_tf_dataset(df, image_dir, tabular_data=None, batch_size=32, is_training=True):

    # Prepare image paths and labels
    image_paths = [os.path.join(image_dir, f"{isic_id}.jpg") for isic_id in df['isic_id']]
    labels = df['malignant'].values.astype(np.float32)

    def load_and_preprocess_image(image_path, label, tabular_features=None):
        # Load image
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, IMG_SIZE)
        image = tf.cast(image, tf.float32) / 255.0

        # Data augmentation for training (following best practices for medical imaging)
        if is_training:
            # Geometric augmentations
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)

            # Color augmentations (subtle for medical images)
            image = tf.image.random_brightness(image, 0.1)
            image = tf.image.random_contrast(image, 0.9, 1.1)
            image = tf.image.random_saturation(image, 0.9, 1.1)

            # Random rotation (0, 90, 180, 270 degrees)
            k = tf.random.uniform([], 0, 4, dtype=tf.int32)
            image = tf.image.rot90(image, k)

        if tabular_features is not None:
            return {'image_input': image, 'tabular_input': tabular_features}, label
        else:
            return image, label

    # Create dataset
    if tabular_data is not None:
        # Multimodal dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels, tabular_data))
        dataset = dataset.map(
            lambda path, label, tabular: load_and_preprocess_image(path, label, tabular),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    else:
        # Image-only dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
        dataset = dataset.map(
            lambda path, label: load_and_preprocess_image(path, label),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    # Apply batching and prefetching
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# Create datasets
print("Creating training and validation datasets...")

# Image-only datasets
train_dataset_img = create_tf_dataset(train_df, IMAGE_DIR, batch_size=BATCH_SIZE, is_training=True)
val_dataset_img = create_tf_dataset(val_df, IMAGE_DIR, batch_size=BATCH_SIZE, is_training=False)

# Multimodal datasets
train_dataset_multi = create_tf_dataset(train_df, IMAGE_DIR, train_tabular, batch_size=BATCH_SIZE, is_training=True)
val_dataset_multi = create_tf_dataset(val_df, IMAGE_DIR, val_tabular, batch_size=BATCH_SIZE, is_training=False)

print("✓ Data pipelines created with augmentation")

Creating training and validation datasets...
✓ Data pipelines created with augmentation


In [27]:
#training with callbacks

# Setup callbacks for training
callbacks = [
    EarlyStopping(
        monitor='val_auc',
        patience=7,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_efficientnet_model.h5',
        monitor='val_auc',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

print(" Training EfficientNet-B0 CNN (Image-only)...")

try:
    history_cnn = cnn_model.fit(
        train_dataset_img,
        validation_data=val_dataset_img,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
    print(" EfficientNet-B0 CNN training completed!")

except Exception as e:
    print(f" Error during CNN training: {e}")
    print("This might be due to memory constraints or missing images.")

print("\n Training Multimodal EfficientNet-B0...")

try:
    history_multimodal = multimodal_cnn.fit(
        train_dataset_multi,
        validation_data=val_dataset_multi,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
    print(" Multimodal EfficientNet-B0 training completed!")

except Exception as e:
    print(f" Error during multimodal training: {e}")

 Training EfficientNet-B0 CNN (Image-only)...
Epoch 1/30
 Error during CNN training: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Error in user-defined function passed to ParallelMapDatasetV2:21 transformation with iterator: Iterator::Root::Prefetch::BatchV2::Shuffle::ParallelMapV2: /Users/raielamesquita/Desktop/ISIC_2024_Training_Input/ISIC_7147422.jpg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_68397]
This might be due to memory constraints or missing images.

 Training Multimodal EfficientNet-B0...
Epoch 1/30
 Error during multimodal training: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Error in user-defined function passed to ParallelMapDatasetV2:30 transformation with iterator: Iterator::Root::Prefetch::BatchV2::Shuffle::ParallelMapV2: /Users/raielamesquita/Desktop/ISIC_2024_Trai

In [31]:
#evaluation

def evaluate_clinical_performance(model, val_dataset, val_df, model_name):


    # Get predictions
    y_pred_probs = model.predict(val_dataset)
    y_pred_binary = (y_pred_probs >= 0.5).astype(int).flatten()
    y_true = val_df['malignant'].values

    # Clinical metrics
    auc_score = roc_auc_score(y_true, y_pred_probs)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    # Clinical metrics
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall for malignant
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Recall for benign
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1_score = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    print(f"AUC-ROC: {auc_score:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score: {f1_score:.4f}")

    print(f"\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"Actual     Benign  Malignant")
    print(f"Benign     {tn:6d}  {fp:9d}")
    print(f"Malignant  {fn:6d}  {tp:9d}")

    # Clinical interpretation
    print(f"\n Clinical Interpretation:")
    if sensitivity >= 0.90:
        print(f"    Excellent sensitivity ({sensitivity:.1%}) - Very few malignant cases missed")
    elif sensitivity >= 0.80:
        print(f"    Good sensitivity ({sensitivity:.1%}) - Acceptable for screening")
    else:
        print(f"    Low sensitivity ({sensitivity:.1%}) - May miss malignant cases")

    if specificity >= 0.90:
        print(f"    Excellent specificity ({specificity:.1%}) - Low false positive rate")
    elif specificity >= 0.80:
        print(f"    Good specificity ({specificity:.1%}) - Acceptable false positive rate")
    else:
        print(f"     Low specificity ({specificity:.1%}) - High false positive rate")

    return {
        'auc': auc_score,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1_score': f1_score
    }

# Evaluate both models
try:
    cnn_results = evaluate_clinical_performance(cnn_model, val_dataset_img, val_df, "EfficientNet-B0 CNN")
    multimodal_results = evaluate_clinical_performance(multimodal_cnn, val_dataset_multi, val_df, "Multimodal EfficientNet-B0")

    # Final comparison
    print(f"\n FINAL MODEL COMPARISON")
    print("="*60)
    print(f"{'Metric':<15} {'CNN Only':<12} {'Multimodal':<12} {'Improvement':<12}")
    print("-"*60)
    print(f"{'AUC-ROC':<15} {cnn_results['auc']:<12.4f} {multimodal_results['auc']:<12.4f} {multimodal_results['auc']-cnn_results['auc']:+.4f}")
    print(f"{'Sensitivity':<15} {cnn_results['sensitivity']:<12.4f} {multimodal_results['sensitivity']:<12.4f} {multimodal_results['sensitivity']-cnn_results['sensitivity']:+.4f}")
    print(f"{'Specificity':<15} {cnn_results['specificity']:<12.4f} {multimodal_results['specificity']:<12.4f} {multimodal_results['specificity']-cnn_results['specificity']:+.4f}")
    print(f"{'F1-Score':<15} {cnn_results['f1_score']:<12.4f} {multimodal_results['f1_score']:<12.4f} {multimodal_results['f1_score']-cnn_results['f1_score']:+.4f}")

except Exception as e:
    print(f" Error during evaluation: {e}")

print(f"\n EfficientNet-B0 CNN Implementation Complete!")
print(f" Best model saved as 'best_efficientnet_model.h5'")

 Error during evaluation: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /Users/raielamesquita/Desktop/ISIC_2024_Training_Input/ISIC_9114250.jpg; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

 EfficientNet-B0 CNN Implementation Complete!
 Best model saved as 'best_efficientnet_model.h5'
